In [1]:
import pandas as pd

In [13]:
import toolkit
import dotenv

dotenv.load_dotenv()

True

In [11]:
raw_rental_data_path = "datasets/raw/rental_data.csv"
raw_data = pd.read_csv(raw_rental_data_path,encoding="latin")
raw_data['rent per month in euro'] = raw_data['rent per month in euro'].map(toolkit.getprice)
raw_data['rent per annum in euro'] = raw_data['rent per annum in euro'].map(toolkit.getprice)



raw_data['size range (sq ft)'] = raw_data['size range (sq ft)'].map(toolkit.gerrange)
raw_data['size range (sq m)'] = raw_data['size range (sq m)'].map(toolkit.gerrange)
raw_data.head()

,address,City,rent per month in euro,rent per annum in euro,size range (sq ft),size range (sq m),key features,description,link
0,"55 Goswell Road, Clerkenwell, EC1V 7EN",London,3255,39060,601-2768,56-257,Newly Refurbished studio officesFully fitted a...,"The building is located on Goswell Road, a pri...",https://www.rightmove.co.uk/properties/1251163...
1,"415 Wick Lane, E3",London,3850,46200,849-545000,79-50632,Flexible leasesCommunity manager and reception...,Studio G101: £3850 pcm 8% service charge 849 s...,https://www.rightmove.co.uk/properties/1457024...
2,"4-10 North Road, Islington",London,5688,68256,871-535000,81-49703,Flexible leasesCommunity manager and reception...,"Studio 0.07: £5,688 pcm Size: 871 sq/ft Reserv...",https://www.rightmove.co.uk/properties/1497522...
3,"Republic, Import Building, E14 2BE",London,2917,35004,1000-400000,93-37161,Dedicated On-site Events TeamFantastic Amentie...,A state of independence for an independent sta...,https://www.rightmove.co.uk/properties/7959716...
4,"30 South Colonnade, London, E14 5EP",London,7356,88272,1605-302773,149-28129,30 showers300 lockersBike StoreLevels 1 -12,Per sq ft £55.00 per sq ft An iconic gateway w...,https://www.rightmove.co.uk/properties/1173844...


### data-cleanup

In [58]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

template_string = """Address: {address}
City: {city}
Answer: The expanded and cleaned up version of this address without country name but city name included is: """
prompt = PromptTemplate.from_template(template_string)

llm = OpenAI(temperature=0)
llm_chain = prompt | llm


clean_address = lambda address, city: llm_chain.invoke(dict(address=address,city=city)).strip()


In [59]:
updated_addresses = raw_data['address'].map(lambda x: clean_address(x, "London"))
updated_addresses

0       55 Goswell Road, Clerkenwell, London, EC1V 7EN.
1                             415 Wick Lane, London E3.
2                   4-10 North Road, Islington, London.
3             Republic, Import Building, London E14 2BE
4                   30 South Colonnade, London E14 5EP.
5                   4-10 North Road, Islington, London.
6     Chimney Walk, Sugar House Island, Stratford, L...
7                                77 Marsh Wall, London.
8                        1 Clove Crescent, London, E14.
9              151 Buckingham Palace Road, London SW1W.
10                            415 Wick Lane, London E3.
11                  1 Fore Street Avenue, London, EC2Y.
12    60 Whitechapel High Street, Whitechapel, Londo...
13                            415 Wick Lane, London E3.
14                   60 New Broad Street, London, EC2M.
15                           415 Wick Lane, London, E3.
16                           415 Wick Lane, London, E3.
17                    High Street, Croydon, Lond

In [60]:
raw_data['address'] = updated_addresses

In [99]:
import json
template_string = """Key Features: `{features}`
Answer: The key features of office rental properties given elegantly split up by lines and cleaned up is following.
make sure the output is in json array.
"""
prompt = PromptTemplate.from_template(template_string)

llm = OpenAI(temperature=0)
llm_chain = prompt | llm


clean_key_features = lambda features: " ".join(json.loads(llm_chain.invoke(dict(features=features)).strip()))


In [100]:
key_featues_clean = raw_data['key features'].map(lambda x: [print(x), print(w:=clean_key_features(x)), w][2])


This building is located in a prime Clerkenwell location on Goswell Road. The studios have been recently refurbished to a high standard, with modern suspended lighting, wooden flooring, and BT fibre connectivity. Many studios and headline events for Clerkenwell Design Week are located here. The area is home to major companies in design, fashion, technology, and showrooms. Transport is easily accessible, with Farringdon station serving multiple lines and the Cycle Superhighway nearby. The floors are offered fully fitted and furnished, with the option for a fully managed space. The property is elected for VAT and can be leased directly from the freeholder.
This building is located in a prime Clerkenwell location on Goswell Road. The studios have been recently refurbished to a high standard, with modern suspended lighting, wooden flooring, and BT fibre connectivity. Many studios and headline events for Clerkenwell Design Week are located here. The area is home to major companies in design

In [101]:
raw_data['key features'] = key_featues_clean

In [103]:

template_string = """Description: `{description}`
Answer: The cleaned up and elegant and summarized description of is following. 
"""
prompt = PromptTemplate.from_template(template_string)

llm = OpenAI(temperature=0)
llm_chain = prompt | llm


clean_description = lambda description: llm_chain.invoke(dict(description=description)).strip()


In [104]:
description_clean = raw_data['description'].map(lambda x: [print(x,end="\n\n"), print(w:=clean_description(x), end="\n\n"), w][2])


This building is located in a prime Clerkenwell location on Goswell Road. The studios have been recently refurbished to a high standard, with modern suspended lighting, wooden flooring, and BT fibre connectivity. Many studios and headline events for Clerkenwell Design Week are located here. The area is home to major companies in design, fashion, technology, and showrooms. Transport is easily accessible, with Farringdon station serving multiple lines and the Cycle Superhighway nearby. The floors are offered fully fitted and furnished, with the option for a fully managed space. The property is elected for VAT and can be leased directly from the freeholder.

This building is situated in a highly desirable location in Clerkenwell, on Goswell Road. The recently renovated studios feature modern suspended lighting, wooden flooring, and BT fibre connectivity. It is a popular spot for Clerkenwell Design Week and is surrounded by major companies in design, fashion, and technology. Farringdon sta

In [105]:
raw_data['description']=description_clean

In [108]:
raw_data

,address,City,rent per month in euro,rent per annum in euro,size range (sq ft),size range (sq m),key features,description,link
0,"55 Goswell Road, Clerkenwell, London, EC1V 7EN.",London,3255,39060,601-2768,56-257,This building is located in a prime Clerkenwel...,This building is situated in a highly desirabl...,https://www.rightmove.co.uk/properties/1251163...
1,"415 Wick Lane, London E3.",London,3850,46200,849-545000,79-50632,Studio G101 is a loft-style studio located in ...,Studio G101 is a versatile loft-style studio l...,https://www.rightmove.co.uk/properties/1457024...
2,"4-10 North Road, Islington, London.",London,5688,68256,871-535000,81-49703,Studio 0.07 is a Cowarehousing space located i...,Studio 0.07 is a Cowarehousing space located i...,https://www.rightmove.co.uk/properties/1497522...
3,"Republic, Import Building, London E14 2BE",London,2917,35004,1000-400000,93-37161,Newly transformed contemporary campus in East ...,The Import Building is a modern campus in East...,https://www.rightmove.co.uk/properties/7959716...
4,"30 South Colonnade, London E14 5EP.",London,7356,88272,1605-302773,149-28129,"Located in the iconic Canary Wharf, this prope...","""Situated in the renowned Canary Wharf, this p...",https://www.rightmove.co.uk/properties/1173844...
5,"4-10 North Road, Islington, London.",London,1495,17940,195-240000,18-22297,Studio sizes ranging from 195 sq/ft to 1000 sq...,Tradestars Islington offers a unique concept o...,https://www.rightmove.co.uk/properties/1500277...
6,"Chimney Walk, Sugar House Island, Stratford, L...",London,19977,239724,5993-216771,557-20139,"216,771 Sq Ft of new build offices Roof terrac...","216,771 Sq Ft of new build offices with roof t...",https://www.rightmove.co.uk/properties/1306062...
7,"77 Marsh Wall, London.",London,250,3000,100-210000,9-19510,"Newly renovated 210,000 sq ft building Located...",Sierra Quebec Bravo is a recently renovated 21...,https://www.rightmove.co.uk/properties/1346730...
8,"1 Clove Crescent, London, E14.",London,2917,35004,1000-180000,93-16723,The Export Building is a newly transformed con...,The Export Building is a modern campus in East...,https://www.rightmove.co.uk/properties/7959885...
9,"151 Buckingham Palace Road, London SW1W.",London,11435,137220,2111-175799,196-16332,"Modern, Grade A office space Undergoing a comp...","151 Buckingham Palace Road is a modern, Grade ...",https://www.rightmove.co.uk/properties/1486184...


In [109]:
raw_data.to_csv("datasets/processed/london.csv", index=False)